In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, confusion_matrix, classification_report


from category_encoders import TargetEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import svm


In [69]:
def cross_validation(X,y,model):
    f1 = []
    conv = []
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        f1_macro = f1_score(y_test, y_pred, average = 'macro')
       
        f1.append(f1_macro)
        #conv.append(confusion_matrix(y_test, y_pred))
        print(f'here is the : {f1_macro}')
        print(classification_report(y_test, y_pred))
        print("==="*20)
    return f1, conv

In [3]:
train = pd.read_csv("../data/preprocess/train_2.csv")
test  = pd.read_csv("../data/preprocess/test_2.csv")
train = train.iloc[:,2:]
test = test.iloc[:,2:]

/var/folders/yt/n9_2kr5s6nx_5b2y_kx3mnnm0000gn/T/ipykernel_43565/3530008456.py:1: DtypeWarning: Columns (6,7,8,9,10,11,12,13,14,17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("../data/preprocess/train_2.csv")


In [4]:
train.isna().sum()


floors_before_eq (total)          332806
old_building                      239204
plinth_area (ft^2)                421208
height_before_eq (ft)             332806
land_surface_condition            301606
type_of_foundation                239204
type_of_roof                      421208
type_of_ground_floor              332806
type_of_other_floor               301606
position                          312006
building_plan_configuration       301606
technical_solution_proposed       676014
legal_ownership_status            124802
has_secondary_use                 197604
type_of_reinforcement_concrete    291206
residential_type                  270404
no_family_residing                145602
public_place_type                      0
industrial_use_type               114402
govermental_use_type              249604
flexible_superstructure            62400
wall_binding                       62400
wall_material                     228804
damage_grade                           0
dtype: int64

In [5]:
test.isna().sum()

floors_before_eq (total)          0
old_building                      0
plinth_area (ft^2)                0
height_before_eq (ft)             0
land_surface_condition            0
type_of_foundation                0
type_of_roof                      0
type_of_ground_floor              0
type_of_other_floor               0
position                          0
building_plan_configuration       0
technical_solution_proposed       0
legal_ownership_status            0
has_secondary_use                 0
type_of_reinforcement_concrete    0
residential_type                  0
no_family_residing                0
public_place_type                 0
industrial_use_type               0
govermental_use_type              0
flexible_superstructure           0
wall_binding                      0
wall_material                     0
dtype: int64

In [6]:
for c in train.columns:
    print(f"{c} with dtypes of {train[c].dtypes}")
    print(train[c].nunique())
    print(train[c].unique())
    print(f"There are : {train[c].isna().sum()/train.shape[0] }  missing value")
    print("===="*20)

floors_before_eq (total) with dtypes of float64
9
[ 2.  3. nan  1.  5.  4.  6.  8.  9.  7.]
There are : 0.46043040058659546  missing value
old_building with dtypes of float64
160
[  1.   3.   7.  18.  22.  nan  33.  40.  27.  68.  25.  15.  20.  28.
  35.  10.   4.   8.   5.  12.  32.  45.  13.  50.   6.  36.  70.  30.
  14.  75.  90.  34.  24.   2.   9.  16.  29. 999.  67.  60.  26.  38.
   0.  17.  37.  19.  80.  21.  11.  55.  44.  39.  23.  65.  48.  85.
  52.  31. 100.  57.  42.  41.  62. 120.  64.  43.  49.  47.  46.  88.
  82.  87.  69. 105.  99.  54. 140.  51.  72.  95.  63. 150. 176.  59.
  81.  58.  61.  53.  56.  84. 110.  66.  74.  98.  86. 109. 111. 200.
  73. 103.  79. 112.  78.  97.  92. 160.  71. 180.  76.  91.  96. 101.
 166. 106. 118. 108.  83.  77. 145. 130. 117.  93. 102. 135. 104. 170.
 131. 115.  89. 125. 128. 119.  94. 190. 174. 126. 132. 113. 116. 144.
 162. 138. 195. 141. 122. 175. 161. 196. 107. 121. 151. 142. 177. 124.
 127. 134. 178. 133. 123. 172. 168.]
The

In [7]:
for c in test.columns:
    print(f"{c} with dtypes of {test[c].dtypes}")
    print(test[c].nunique())
    print(test[c].unique())
    print(f"There are : {test[c].isna().sum()/test.shape[0] }  missing value")
    print("===="*20)

floors_before_eq (total) with dtypes of float64
8
[2. 3. 1. 5. 4. 6. 7. 9.]
There are : 0.0  missing value
old_building with dtypes of int64
144
[  7  13  40  25   5   6  10  17  31  18  44   2  15  16  12  30   3 999
  21   4  11   9  22  35  23  20  27   1   8  14  47  34  42  50 100  19
  32  33  64  28  60  45   0  29  70  26  38  41  39  54  24  37  80  43
  36  56  58  55  62 200  76  68  51  48  78  99  46  82  90  65 110 140
  52  75  63  57  85  83  81  61  53 114  71  95  72 102  94  69  74  73
  49 120 123  66  59 150 125  86  67 103  79 115  84  77 112 105 116  92
  88 122 130 104  96 109  89  97 111  87 107  98  91 148 160 180 113 108
 156 141 155 133 170 118 135 175 172 178 152 190 101 151 149  93 117 195]
There are : 0.0  missing value
plinth_area (ft^2) with dtypes of float64
928
[ 418.  396.  400.  378.  375.  534.  416.  558.  255.  300.  216.  342.
  457.  350.  240.  310.  450.  672.  260.  308. 1001.  270.  162.  150.
  891.  648.  180.  420.  200.  275.  392.  695

In [8]:
train

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,2.0,1.0,256.0,22.0,Flat,bamboo/timber,bamboo/timber light roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,0.0,0.0,1.0
1,3.0,3.0,985.0,18.0,Flat,clay mortar-stone/brick,wood light roof or bamboo heavy roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
2,2.0,7.0,NaN,14.0,Flat,mud mortar-stone/brick,NaN,clay,wood or bamboo mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,18.0,185.0,15.0,Flat,clay mortar-stone/brick,wood light roof or bamboo light roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
4,2.0,22.0,290.0,17.0,Flat,clay mortar-stone/brick,bamboo/timber light roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722810,2.0,32.0,NaN,12.0,Steep slope,clay mortar-stone/brick,NaN,clay,wood or bamboo mud,Attached-1 side,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,1.0,0.0,4.0
722811,3.0,45.0,NaN,18.0,Moderate slope,clay mortar-stone/brick,NaN,mud,lumber-plank,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
722812,3.0,72.0,NaN,21.0,Flat,clay mortar-stone/brick,NaN,clay,timber/bamboo-mud,Attached-1 side,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,1.0,5.0
722813,1.0,22.0,NaN,6.0,Flat,clay mortar-stone/brick,NaN,clay,not applicable,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0


In [71]:
# data = train.copy().drop('technical_solution_proposed',axis=1)
# data = data.dropna(thresh=15)

# print(data.shape)
# X = data.iloc[:,:-1]
# y = data.iloc[:,-1]

# ### train
# cat_features = X .select_dtypes(include='object').columns.tolist()
# one_hot_encoded = pd.get_dummies(X[cat_features])
# cols_var = [i for i in X.columns if i not in cat_features]
# tmp = (X[cols_var]-X[cols_var].mean())/X[cols_var].std()
# tmp = pd.concat([tmp,X[cat_features]],axis=1)

# tmp = tmp.apply(lambda series: pd.Series(
#         LabelEncoder().fit_transform(series[series.notnull()]),
#         index=series[series.notnull()].index
#     ))
# print(tmp.isna().sum())
# imputer = KNNImputer(n_neighbors=1)
# tmp = imputer.fit_transform(tmp)

# df = pd.DataFrame(tmp,columns=X.columns)

In [72]:
train

,floors_before_eq (total),old_building,plinth_area (ft^2),height_before_eq (ft),land_surface_condition,type_of_foundation,type_of_roof,type_of_ground_floor,type_of_other_floor,position,...,type_of_reinforcement_concrete,residential_type,no_family_residing,public_place_type,industrial_use_type,govermental_use_type,flexible_superstructure,wall_binding,wall_material,damage_grade
0,2.0,1.0,256.0,22.0,Flat,bamboo/timber,bamboo/timber light roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,0.0,0.0,1.0
1,3.0,3.0,985.0,18.0,Flat,clay mortar-stone/brick,wood light roof or bamboo heavy roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
2,2.0,7.0,NaN,14.0,Flat,mud mortar-stone/brick,NaN,clay,wood or bamboo mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
3,2.0,18.0,185.0,15.0,Flat,clay mortar-stone/brick,wood light roof or bamboo light roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,4.0
4,2.0,22.0,290.0,17.0,Flat,clay mortar-stone/brick,bamboo/timber light roof,clay,timber/bamboo-mud,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
722810,2.0,32.0,NaN,12.0,Steep slope,clay mortar-stone/brick,NaN,clay,wood or bamboo mud,Attached-1 side,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,1.0,0.0,4.0
722811,3.0,45.0,NaN,18.0,Moderate slope,clay mortar-stone/brick,NaN,mud,lumber-plank,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0
722812,3.0,72.0,NaN,21.0,Flat,clay mortar-stone/brick,NaN,clay,timber/bamboo-mud,Attached-1 side,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,1.0,5.0
722813,1.0,22.0,NaN,6.0,Flat,clay mortar-stone/brick,NaN,clay,not applicable,Not attached,...,0.0,Non-residential,1.0,Non-public,Non-industrial,Non-govermental,unavailable,5.0,2.0,5.0


In [47]:
te = TargetEncoder(cols=obj_cols)
y_ = y.iloc[df.index]
y_.index = df.index
df_t = te.fit_transform(df,y_)

In [74]:
cross_validation(df_t,y_,RandomForestClassifier(random_state=0))

here is the : 0.44677513218898757
              precision    recall  f1-score   support

         1.0       0.58      0.61      0.59      8753
         2.0       0.41      0.27      0.32      9625
         3.0       0.39      0.27      0.32     15105
         4.0       0.42      0.37      0.39     20295
         5.0       0.53      0.70      0.61     30464

    accuracy                           0.49     84242
   macro avg       0.46      0.44      0.45     84242
weighted avg       0.47      0.49      0.47     84242

here is the : 0.44683231998323547
              precision    recall  f1-score   support

         1.0       0.57      0.61      0.59      8753
         2.0       0.41      0.26      0.32      9625
         3.0       0.39      0.28      0.33     15105
         4.0       0.42      0.36      0.39     20296
         5.0       0.53      0.70      0.61     30463

    accuracy                           0.49     84242
   macro avg       0.47      0.44      0.45     84242
weighted 

([0.44677513218898757,
  0.44683231998323547,
  0.4451551359784573,
  0.44286376253360515,
  0.4428655156090711],
 [])